In [1]:
from compas.geometry import Frame, Plane
from compas.geometry import Box, Sphere
from compas_vol.primitives import VolBox, VolSphere
from compas_vol.combinations import Union, SmoothUnion, Intersection
from compas_vol.modifications import Shell

In [2]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import get_random_vector_3D, bbox_edges

In [3]:
#generate random points around a sphere
nbrPts = 10
sphereRad = 3.4
pts = [get_random_vector_3D(sphereRad) for _ in range(nbrPts)]
print(pts)

#u = np.random.normal(size=nbrPts)
#v = np.random.normal(size=nbrPts)
#w = np.random.normal(size=nbrPts)
#norm = (u*u + v*v + w*w)**(0.5)
#pts = np.array([u/norm, v/norm, w/norm]).T * sphereRad

[(-1.7286968178409143, -2.925471349506095, -0.11499867479968717), (-2.3624861462253874, 0.41738695434016465, -2.4092420673813075), (-3.153882018648314, -0.8188694943728675, 0.9708145876697366), (1.405722249988638, 2.0349770203164015, -2.3329838153470037), (0.9564251570563488, -1.1517657355283182, 3.05265238922034), (0.7311875504395597, 3.262801197934418, -0.6160301200752467), (-3.3274096409580376, -0.6817607380881366, -0.15345154694894073), (-1.8311149653771375, 2.516333185756442, 1.3693375339311769), (2.491704692887145, -1.2404618146584165, 1.9526039562129656), (0.6587968962061715, 2.575145139969862, -2.120050508275384)]


In [4]:
#create volumetric boxes and unify them
volBoxes = []
for [x,y,z] in pts:
    plane = Plane([x,y,z],[x,y,z])
    box = Box(Frame.from_plane(plane),1.2,1.2,3.5)
    vbox = VolBox(box, 0.1)
    volBoxes.append(vbox)
uvb = Union(volBoxes)

In [5]:
#combine core and branches
vs = VolSphere(Sphere([0,0,0], 2.0))
su = SmoothUnion(uvb, vs, 2.0)

In [6]:
#thicken shell and cull extremities
s = Shell(su, 0.2, 1.0)
node = Intersection(VolSphere(Sphere([0,0,0], 4.8)), s)

In [7]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -5.0, 5.0
lby, uby = -5.0, 5.0
lbz, ubz = -5.0, 5.0
# resolution(s)
nx, ny, nz = 100, 100, 100
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

In [8]:
dm = node.get_distance_numpy(x, y, z)

In [9]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

In [10]:
p = mp.plot(v, f, c=np.array([0,0.57,0.82]), shading={"flat":False, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})

/Users/bernham/anaconda3/envs/compasdev/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.447141…